In [44]:
import cv2
import numpy as np

In [53]:
def filter2d(image ,kernel):
    height, width, channels = image.shape
    k_height, k_width = kernel.shape[:2]
    
    pad_height = k_height // 2
    pad_width = k_width // 2
    
    output_image = np.zeros((height, width, channels), dtype=np.uint8)
    
    for c in range(channels):
        for i in range(pad_height, height - pad_height):
            for j in range(pad_width, width - pad_width):
                neighborhood = image[i - pad_height:i + pad_height + 1, j - pad_width:j + pad_width + 1, c]
                filtered_pixel = np.sum(neighborhood * kernel)
                output_image[i, j, c] = filtered_pixel
    
    return output_image


In [54]:
def gaussian_blur(image, kernel_size=5, sigma=1.5):

    # Check if the image is loaded successfully
    if image is None:
        print("Error: Unable to read the image.")
        return None

    # Convert the image to float32 for numerical stability
    image = image.astype(np.float32)

    # Create a Gaussian kernel
    kernel = np.fromfunction(
        lambda x, y: (1 / (2 * np.pi * sigma ** 2)) *
                    np.exp(-((x - kernel_size // 2) ** 2 + (y - kernel_size // 2) ** 2) / (2 * sigma ** 2)),
        (kernel_size, kernel_size)
    )

    # Normalize the kernel
    kernel = kernel / kernel.sum()

    # Apply the Gaussian blur by convolving the image with the kernel
    blurred_image = filter2d(image, kernel)

    # Convert back to uint8 for display
    blurred_image = blurred_image.astype(np.uint8)

    return blurred_image

In [55]:
def sobel_operator(image):
    # Sobel kernels
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    # Convolve the image with Sobel kernels
    gradient_x = convolve(image, sobel_x)
    gradient_y = convolve(image, sobel_y)

    # Compute gradient magnitude and direction
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_direction = np.arctan2(gradient_y, gradient_x)

    return gradient_magnitude, gradient_direction

def convolve(image, kernel):
    height, width = image.shape
    k_height, k_width = kernel.shape

    # Calculate the padding to handle the kernel size
    pad_height = k_height // 2
    pad_width = k_width // 2

    # Create an empty output image
    output_image = np.zeros((height, width))

    # Iterate over the input image and apply convolution
    for i in range(pad_height, height - pad_height):
        for j in range(pad_width, width - pad_width):
            neighborhood = image[i - pad_height:i + pad_height + 1, j - pad_width:j + pad_width + 1]
            output_image[i, j] = np.sum(neighborhood * kernel)

    return output_image

def canny_edge_detection(image, low_threshold, high_threshold):

    # Step 2: Compute gradient using Sobel operators
    gradient_magnitude, gradient_direction = sobel_operator(image)

    # Step 3: Non-maximum suppression
    suppressed_image = non_maximum_suppression(gradient_magnitude, gradient_direction)

    # Step 4: Edge tracking by hysteresis
    edges = hysteresis_thresholding(suppressed_image, low_threshold, high_threshold)

    return edges.astype(np.uint8) * 255

def non_maximum_suppression(magnitude, direction):
    height, width = magnitude.shape
    suppressed = np.zeros_like(magnitude)

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            angle = direction[i, j]

            # Define gradient directions
            grad_dir = (0 if (angle < np.pi / 8) or (angle >= 7 * np.pi / 8) else
                        1 if (angle >= np.pi / 8) and (angle < 3 * np.pi / 8) else
                        2 if (angle >= 3 * np.pi / 8) and (angle < 5 * np.pi / 8) else
                        3)

            # Perform non-maximum suppression
            if grad_dir == 0:
                neighbors = (magnitude[i, j - 1], magnitude[i, j + 1])
            elif grad_dir == 1:
                neighbors = (magnitude[i - 1, j - 1], magnitude[i + 1, j + 1])
            elif grad_dir == 2:
                neighbors = (magnitude[i - 1, j], magnitude[i + 1, j])
            else:
                neighbors = (magnitude[i - 1, j + 1], magnitude[i + 1, j - 1])

            suppressed[i, j] = magnitude[i, j] if magnitude[i, j] >= max(neighbors) else 0

    return suppressed

def hysteresis_thresholding(image, low_threshold, high_threshold):
    strong_edges = (image >= high_threshold)
    weak_edges = (image >= low_threshold) & (image < high_threshold)

    # Perform edge tracking by hysteresis
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            if weak_edges[i, j]:
                # Check if any of the 8 neighboring pixels is a strong edge
                if np.any(strong_edges[i-1:i+2, j-1:j+2]):
                    strong_edges[i, j] = True
                else:
                    weak_edges[i, j] = False

    return strong_edges.astype(np.uint8)

In [58]:
import cv2
import numpy as np

def resize_frame(frame, scale_percent):
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    return cv2.resize(frame, (width, height))

# Load the video
video = cv2.VideoCapture(r'C:\Users\kapitoo\Downloads\feter_meshaltet.mp4')

# Get the video properties (width, height, frames per second)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Changed to mp4
out = cv2.VideoWriter(r'C:\Users\kapitoo\Downloads\output_video.mp4', fourcc, fps, (width, height))

while True:
    ret, orig_frame = video.read()

    if not ret:
        video = cv2.VideoCapture(r'C:\Users\kapitoo\Downloads\feter_meshaltet.mp4')
        continue

    # Resize frame for faster processing
    scale_percent = 50  # Adjust as needed
    frame = resize_frame(orig_frame, scale_percent)

    # Apply Gaussian blur
    frame = cv2.GaussianBlur(frame, (5, 5), 0)

    # Convert frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Adjusted HSV values for yellow
    low_yellow = np.array([15, 50, 150])
    up_yellow = np.array([35, 255, 255])

    # Adjusted HSV values for white
    low_white = np.array([0, 0, 200])
    up_white = np.array([255, 30, 255])

    # Create masks for yellow and white
    mask_yellow = cv2.inRange(hsv, low_yellow, up_yellow)
    mask_white = cv2.inRange(hsv, low_white, up_white)

    # Combine the masks
    combined_mask = cv2.bitwise_or(mask_yellow, mask_white)

    # Set the region of interest (ROI) to a slightly lower portion of the image
    height, width = frame.shape[:2]
    roi_mask = np.zeros_like(combined_mask)
    roi_mask[int(0.6 * height):height, :] = 255
    combined_mask = cv2.bitwise_and(combined_mask, roi_mask)

    # Apply Canny edge detection
    edges = cv2.Canny(combined_mask, 50, 150)  # Adjust the thresholds

    # Detect lines using Hough transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=100)

    # Draw lines on the frame
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)

    # Write the frame to the output video
    out.write(frame)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Exit if the user presses 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release the VideoWriter and VideoCapture objects
out.release()
video.release()
cv2.destroyAllWindows()
